In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/Train.csv', sep=';')
test = pd.read_csv('./data/TestX.csv', sep=';')

In [3]:
train = train.drop_duplicates('opinion')

In [4]:
results_df = pd.merge(test, train, on=['opinion', 'condition'], how='left')

In [5]:
results_df

,name_x,condition,opinion,name_y,rate,rate1
0,Vitamin B12,Vitamin B12 Deficiency,"""No after taste at unlike all the others that ...",Cyanocobalamin,10.0,high
1,Implanon,Birth Control,"""I have not had any issues with the implanon I...",Etonogestrel,8.0,high
2,BuSpar,Anxiety,"""I have been taking Buspar and Celexa and as f...",Buspirone,10.0,high
3,Abreva,Herpes Simplex,"""the only thing that works. completely awesome.""",Docosanol,10.0,high
4,Buprenorphine / naloxone,Opiate Dependence,"""UPDATE: I originally posted on Nov 16th 2010 ...",Suboxone,10.0,high
...,...,...,...,...,...,...
49995,Acetaminophen / hydrocodone,Pain,"""These just don&#039;t help me at all and they...",NaN,NaN,NaN
49996,Minastrin 24 Fe,Birth Control,"""I got on Minastrin, because I used to take Lo...",Ethinyl estradiol / norethindrone,1.0,low
49997,Yasmin,Acne,"""I am 20 and have had terrible skin since I wa...",NaN,NaN,NaN
49998,Amoxicillin,Bacterial Endocarditis Prevention,"""Very good.""",NaN,NaN,NaN


In [12]:
idx_nan = results_df[results_df['rate'].isna()].index

In [17]:
results_df.iloc[idx_nan, 5]

5        NaN
7        NaN
12       NaN
18       NaN
19       NaN
        ... 
49990    NaN
49995    NaN
49997    NaN
49998    NaN
49999    NaN
Name: rate1, Length: 22750, dtype: object

# rate

In [27]:
import os

In [19]:
train_files = [i for i in os.listdir('results/') if '_val_reg' in i]
test_files = [i for i in os.listdir('results/') if '_test_reg' in i]

train_files.remove('y_val_reg')
train_files = sorted(train_files)
test_files = sorted(test_files)

In [20]:
test_reg = pd.DataFrame({"var" + str(i) : np.fromfile("results/" + file, dtype='float32') for i, file in enumerate(test_files)})

In [24]:
rate = np.round(np.clip(test_reg, 1, 10).mean(axis=1)).astype(int)

# rate1

In [26]:
from tensorflow.keras.utils import to_categorical

In [28]:
def voted_result(filepath, n_models, n_classes):
    
    y_hat_ensemble = None
    
    for i in range(n_models):
        file = filepath.replace('#', str(i))
        y_hat = np.fromfile(file, dtype='float32').reshape(-1, n_classes)
        print(np.argmax(y_hat, axis=1))
        y_hat = to_categorical(np.argmax(y_hat, axis=1), num_classes=n_classes)
            
        if y_hat_ensemble is None:
            y_hat_ensemble = np.zeros_like(y_hat)
        y_hat_ensemble = y_hat_ensemble + y_hat
    
    y_hat_ensemble = y_hat_ensemble + np.random.normal(size=y_hat_ensemble.shape) / 1e4
    
    return np.argmax(y_hat_ensemble, axis=1)

In [29]:
y_test = voted_result('./results/Model_1_#_test_cls', 10, 3)

[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [30]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [66]:
test_clf = np.zeros_like(y_test).astype(str)

test_clf[y_test == 0] = 'high'
test_clf[y_test == 1] = 'low'
test_clf[y_test == 2] = 'medium'

In [67]:
rate1 = pd.Series(test_clf)

In [68]:
rate1

0        high
1        high
2        high
3        high
4        high
         ... 
49995     low
49996     low
49997    high
49998    high
49999    high
Length: 50000, dtype: object

# Final file

In [69]:
results_df.iloc[idx_nan, 4] = rate[idx_nan]

In [70]:
results_df.iloc[idx_nan, 5] = rate1[idx_nan]

In [71]:
results_df.iloc[:, 4] = results_df.iloc[:, 4].astype(int)

In [72]:
results_df

,name_x,condition,opinion,name_y,rate,rate1
0,Vitamin B12,Vitamin B12 Deficiency,"""No after taste at unlike all the others that ...",Cyanocobalamin,10,high
1,Implanon,Birth Control,"""I have not had any issues with the implanon I...",Etonogestrel,8,high
2,BuSpar,Anxiety,"""I have been taking Buspar and Celexa and as f...",Buspirone,10,high
3,Abreva,Herpes Simplex,"""the only thing that works. completely awesome.""",Docosanol,10,high
4,Buprenorphine / naloxone,Opiate Dependence,"""UPDATE: I originally posted on Nov 16th 2010 ...",Suboxone,10,high
...,...,...,...,...,...,...
49995,Acetaminophen / hydrocodone,Pain,"""These just don&#039;t help me at all and they...",NaN,1,low
49996,Minastrin 24 Fe,Birth Control,"""I got on Minastrin, because I used to take Lo...",Ethinyl estradiol / norethindrone,1,low
49997,Yasmin,Acne,"""I am 20 and have had terrible skin since I wa...",NaN,10,high
49998,Amoxicillin,Bacterial Endocarditis Prevention,"""Very good.""",NaN,9,high


In [76]:
results_df[['rate', 'rate1']].to_csv('KRZSPA.csv', index=False, header=True, sep=" ")

In [77]:
import csv

In [ ]:
results_df[['rate', 'rate1']].to_csv('KRZSPA.csv', index=False, header=True, sep=" ", csv=)